In [1]:
include("NTKernel.jl");

In [2]:
# PSUDEO data
g = x -> cos(2x)+sin(x/2)           # anonymous function g(x)

X = range(1, stop=4, length=100)
X = Matrix(reshape(X, 1, :))        # reshape, each column vector is read as data point

Y = map(g,X);                       # Map with g(x)

In [3]:
# nodes on hidden layer
Nh = 100
InputDim = 1
activation = sigmoid

model = Chain(Dense(InputDim=>Nh,activation),Dense(Nh=>InputDim))|>f64

K = kernel(model, X)
normalized_K = K / sqrt(Nh) # Normalize the kernel matrix

λ_values = eigen(normalized_K).values;

In [26]:
Ý = map_model(model,X) # Evaluates all X points, and takes dims into account 😍

In [12]:
using Plots

# The subplot simplified
function graph(model, x, y, λ, logplot=false) # specify if logplotted eigenvalues
    ý = map_model(model, x)

    # Create subplots
    plot1 = plot(x[:], y[:], label="Real data", xlabel="X", ylabel="Y", yaxis=:identity)
    plot!(x[:], ý[:], label="Estimated data")

    # Conditional y-axis scaling based on logplot
    plot2 = scatter(1:length(λ), λ, label="Eigenvalue", xlabel="Id. Eigenvalue", ylabel="Eigenvalue", yaxis=logplot ? :log10 : :identity)

    # Combine subplots into a single plot
    plot(plot1, plot2)
end 

graph(model,X,Y,λ_values)    # Could be graph(model,X,Y,λ_values, true) for logplot

┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\axelb\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\axelb\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\axelb\.julia\packages\PlotUtils\jEGKP\src\ticks.jl:191


<h3>Beware!!!</h3> After the yaxis is changed, you shouldn't change it back. Then the plot function seems to stop working. <br>This might be a bug with the Plog package?

<h1> Now, lets start tinkering with the models </h1>

In [12]:
# nodes on hidden layer

function create_model(Ni, Nh, No, N, activation)
    layers = [] # Initialize an empty array without specifying the type of elements
    push!(layers, Dense(Ni, N, activation)) # First hidden layer with N nodes and specified activation
    for _ in 2:Nh
        push!(layers, Dense(N, N, activation)) # Additional Nh-1 hidden layers
    end
    push!(layers, Dense(N, No)) # Output layer without specifying an activation function (defaults to identity)
    model = Chain(layers...)|>f64 # Create the model from the layers
    return model
end


Nh_list = [1, 2, 3] 
H_Layers = [1, 2, 3]
models = []
activation = sigmoid

InputDim = 1

models = []
for Nh in 1:length(Nh_list)
    for Layer in 1:length(H_Layers)
        push!(models, create_model(InputDim,H_Layers[Layer],InputDim,Nh_list[Nh],activation))
    end
end

Kernels = []
for model in models
    push!(Kernels, kernel(model, X))
end

Kernels_Norms = []
# Iterate over each kernel and normalize according to your pattern
for i in 1:length(Kernels)
    # Use mod1 to cycle through Nh_list based on i
    # This approach correctly associates each kernel with its Nh value according to your pattern
    Nh_index = mod1(i, length(Nh_list))
    Nh = Nh_list[Nh_index]
    normalized_K = Kernels[i] / sqrt(Nh)
    push!(Kernels_Norms, normalized_K)
end
λ_values = []
for K in Kernels_Norms
    push!(λ_values, eigen(K).values) # Compute eigenvalues for each kernel and store
end


# loss(a, b) = Flux.Losses.mse(models[1](a), b)
#Flux.train!(loss, Flux.params(models[1]), [(X,Y)], Descent(0.01))

loss (generic function with 1 method)

In [14]:
for i in 1:length(models)
    display(models[i])
end

Chain(
  Dense(1 => 1, σ),                     # 2 parameters
  Dense(1 => 1),                        # 2 parameters
)                   # Total: 4 arrays, 4 parameters, 288 bytes.

Chain(
  Dense(1 => 1, σ),                     # 2 parameters
  Dense(1 => 1, σ),                     # 2 parameters
  Dense(1 => 1),                        # 2 parameters
)                   # Total: 6 arrays, 6 parameters, 432 bytes.

Chain(
  Dense(1 => 1, σ),                     # 2 parameters
  Dense(1 => 1, σ),                     # 2 parameters
  Dense(1 => 1, σ),                     # 2 parameters
  Dense(1 => 1),                        # 2 parameters
)                   # Total: 8 arrays, 8 parameters, 576 bytes.

Chain(
  Dense(1 => 2, σ),                     # 4 parameters
  Dense(2 => 1),                        # 3 parameters
)                   # Total: 4 arrays, 7 parameters, 312 bytes.

Chain(
  Dense(1 => 2, σ),                     # 4 parameters
  Dense(2 => 2, σ),                     # 6 parameters
  Dense(2 => 1),                        # 3 parameters
)                   # Total: 6 arrays, 13 parameters, 488 bytes.

Chain(
  Dense(1 => 2, σ),                     # 4 parameters
  Dense(2 => 2, σ),                     # 6 parameters
  Dense(2 => 2, σ),                     # 6 parameters
  Dense(2 => 1),                        # 3 parameters
)                   # Total: 8 arrays, 19 parameters, 664 bytes.

Chain(
  Dense(1 => 3, σ),                     # 6 parameters
  Dense(3 => 1),                        # 4 parameters
)                   # Total: 4 arrays, 10 parameters, 336 bytes.

Chain(
  Dense(1 => 3, σ),                     # 6 parameters
  Dense(3 => 3, σ),                     # 12 parameters
  Dense(3 => 1),                        # 4 parameters
)                   # Total: 6 arrays, 22 parameters, 560 bytes.

Chain(
  Dense(1 => 3, σ),                     # 6 parameters
  Dense(3 => 3, σ),                     # 12 parameters
  Dense(3 => 3, σ),                     # 12 parameters
  Dense(3 => 1),                        # 4 parameters
)                   # Total: 8 arrays, 34 parameters, 784 bytes.